# PyctureStream Dashboard

**Demo of a reporting Application based on Plotly Dash.**

Implemented by Marcus. Focus was mainly to gain some experience with Dash.

In [1]:
import pandas as pd
import json
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import csv
import plotly.plotly as py
from dash.dependencies import Input, Output
from datetime import datetime as dt
import datetime

In [2]:
with open('events.json') as f:
    events = json.load(f) 

for event in events[:10]:  
    print(event)

{'event_type': 'person', 'timestamp': '2018-03-18 18:25:56.092099', 'camera_id': 'holger_cam'}
{'event_type': 'book', 'timestamp': '2018-03-18 18:25:56.092099', 'camera_id': 'holger_cam'}
{'event_type': 'book', 'timestamp': '2018-03-18 18:25:56.092099', 'camera_id': 'holger_cam'}
{'event_type': 'book', 'timestamp': '2018-03-18 18:25:56.092099', 'camera_id': 'holger_cam'}
{'event_type': 'person', 'timestamp': '2018-03-18 18:26:03.949571', 'camera_id': 'holger_cam'}
{'event_type': 'chair', 'timestamp': '2018-03-18 18:26:03.949571', 'camera_id': 'holger_cam'}
{'event_type': 'person', 'timestamp': '2018-03-18 18:26:11.830546', 'camera_id': 'holger_cam'}
{'event_type': 'person', 'timestamp': '2018-03-18 18:26:15.766710', 'camera_id': 'holger_cam'}
{'event_type': 'person', 'timestamp': '2018-03-18 18:26:23.652514', 'camera_id': 'holger_cam'}
{'event_type': 'person', 'timestamp': '2018-03-18 18:26:31.633683', 'camera_id': 'holger_cam'}


In [3]:
df = pd.DataFrame(events)
df.head(10) 

,camera_id,event_type,timestamp
0,holger_cam,person,2018-03-18 18:25:56.092099
1,holger_cam,book,2018-03-18 18:25:56.092099
2,holger_cam,book,2018-03-18 18:25:56.092099
3,holger_cam,book,2018-03-18 18:25:56.092099
4,holger_cam,person,2018-03-18 18:26:03.949571
5,holger_cam,chair,2018-03-18 18:26:03.949571
6,holger_cam,person,2018-03-18 18:26:11.830546
7,holger_cam,person,2018-03-18 18:26:15.766710
8,holger_cam,person,2018-03-18 18:26:23.652514
9,holger_cam,person,2018-03-18 18:26:31.633683


In [4]:
df['timestamp']=pd.to_datetime(df.timestamp)
df.dtypes

camera_id             object
event_type            object
timestamp     datetime64[ns]
dtype: object

In [5]:
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['week'] = df['timestamp'].dt.week
df['weekday'] = df['timestamp'].dt.weekday_name
df['day'] = df['timestamp'].dt.day
df['just_date'] = df['timestamp'].dt.date
df.head(10) 

,camera_id,event_type,timestamp,hour,minute,year,month,week,weekday,day,just_date
0,holger_cam,person,2018-03-18 18:25:56.092099,18,25,2018,3,11,Sunday,18,2018-03-18
1,holger_cam,book,2018-03-18 18:25:56.092099,18,25,2018,3,11,Sunday,18,2018-03-18
2,holger_cam,book,2018-03-18 18:25:56.092099,18,25,2018,3,11,Sunday,18,2018-03-18
3,holger_cam,book,2018-03-18 18:25:56.092099,18,25,2018,3,11,Sunday,18,2018-03-18
4,holger_cam,person,2018-03-18 18:26:03.949571,18,26,2018,3,11,Sunday,18,2018-03-18
5,holger_cam,chair,2018-03-18 18:26:03.949571,18,26,2018,3,11,Sunday,18,2018-03-18
6,holger_cam,person,2018-03-18 18:26:11.830546,18,26,2018,3,11,Sunday,18,2018-03-18
7,holger_cam,person,2018-03-18 18:26:15.766710,18,26,2018,3,11,Sunday,18,2018-03-18
8,holger_cam,person,2018-03-18 18:26:23.652514,18,26,2018,3,11,Sunday,18,2018-03-18
9,holger_cam,person,2018-03-18 18:26:31.633683,18,26,2018,3,11,Sunday,18,2018-03-18


In [6]:
df['event_type'].value_counts()

book            2563
person           871
chair            250
couch            169
potted plant      55
teddy bear        45
laptop            43
dining table      31
bottle             8
backpack           4
cup                1
Name: event_type, dtype: int64

In [7]:
df.dtypes

camera_id             object
event_type            object
timestamp     datetime64[ns]
hour                   int64
minute                 int64
year                   int64
month                  int64
week                   int64
weekday               object
day                    int64
just_date             object
dtype: object

In [ ]:
#start = datetime.datetime(2018, 1, 1 )
#end = datetime.datetime.now

available_camera_indicators = df['camera_id'].unique() 
av_camera_options=[{'value': i, 'label': i} for i in available_camera_indicators]

available_event_indicators = df['event_type'].unique() 
av_event_options=[{'value': i, 'label': i} for i in available_event_indicators]



app = dash.Dash()
app.scripts.config.serve_locally=True  

app.layout = html.Div(children=[
    html.H1(children='Test Report', style= {'margin':'45px 0px 90px 50px'}), 
    
    html.Div([   
        html.P('Filter Dataset:', style={'margin':'0px 0px 5px 0px'}),
        dcc.Dropdown(
            id='camera-dropdown',
            options=av_camera_options
        ),   

        dcc.Dropdown(
            id='event-dropdown',
            options=av_event_options
        ),

        dcc.Dropdown(
            id='xaxis-dropdown',
            options=[
                {'label': 'minute', 'value': 'minute'},
                {'label': 'weekday', 'value': 'weekday'},
                {'label': 'hour', 'value': 'hour'}
            ],
            value='minute'
        ),
    ], style={          
            'width': '170px',
            'position': 'absolute',
            'right': '10px',
            'top': '10px',
            'z-index': 10,
            'background-color': '#f7f7f7',
            'padding': '10px',
            'border-radius': '13px',
            'box-shadow': '1px 2px 5px rgba(0,0,0,0.5)'
        }),
    
    html.Div([
        html.Div([   
            
            dcc.Graph(
                id='camera-graph-line',
                figure={
                    'data': [{'x':'df.minute', 
                             'y':'df.timestamp.count()',
                             'text':'df.event_type', 
                             'type':'line', 
                             'name': 'IDK', 
                             'mode':'lines',
                             'marker': {'size': 5}
                             }]
                }
            ), 
        ], style={'flex': '1'}),             
        html.Div([
            dcc.Graph(
                id='camera-graph-histogram',
                figure={
                    'data': [{'x':'df.minute',  
                             'y':'df.timestamp.count()',
                             'text':'df.event_type', 
                             'type':'histogram', 
                             'name' : 'IDK', 
                             'mode':'markers', 
                             'className':'col s12 m6 l6'
                            }]
                }

            ),
        ], style={'flex': '1'}),     
        
        html.Div([
            
            dcc.Graph(
                id='camera-graph-pie',
                figure={
                    'data': [
                            {'values':[    #hab ich die Möglichkeit das zu kontrollieren / Testen? Es kommt keine Fehlermeldung der der Graph ist sofort aktualisiert.
                                av_event_options
                            ], 'labels':[av_event_options], 'type':'pie'}

                    ]
                }
            ),
        ], style={'flex': '1'})     
    ], style={'display': 'flex'})   
    

])
    


@app.callback(
    Output('camera-graph-histogram', 'figure'),
    [Input('event-dropdown', 'value'),
     Input('camera-dropdown', 'value'),
     Input('xaxis-dropdown', 'value')])
def update_graph1(selected_dropdown_value1, selected_dropdown_value2, selected_dropdown_value3):
    
    if selected_dropdown_value1 != None:
        df_histogramm = df[df.event_type == selected_dropdown_value1]
    else:df_histogramm = df
    if selected_dropdown_value2 != None:
        df_hisogramm_second_update = df_histogramm[df.camera_id == selected_dropdown_value2]
    else: df_hisogramm_second_update = df_histogramm
    return{
        'data':[go.Scatter(
            x=df_hisogramm_second_update[selected_dropdown_value3],
            y=df_hisogramm_second_update['timestamp'].count(),  
            text=df_hisogramm_second_update['event_type'],
            name= 'IDK',
            type='histogram',
            mode='markers')
        ],
            'layout': {
                'title' : 'Reporting cameras'
            }
    }
    
@app.callback(
    Output('camera-graph-pie', 'figure'),
    [Input('camera-dropdown', 'value')])
def update_graph2(selected_dropdown_value2):
    if selected_dropdown_value2 != None:
        df_graph_pie = df[df.camera_id == selected_dropdown_value2]
    else: df_graph_pie=df
    df_event_types = df_graph_pie.groupby('event_type').count()
    
    return{
        'data':[go.Pie(
            values=df_event_types.iloc[:,0],      
            labels=df_event_types.index.values,   
            text=df_graph_pie['event_type'],
            name= 'IDK',
            hole= .4,
            type='pie')
        ],
            'layout': {
                'title' : 'Reporting cameras-Pie'
            }
    } 


@app.callback(
    Output('camera-graph-line', 'figure'),
    [Input('event-dropdown', 'value'),
     Input('camera-dropdown', 'value')])
def update_graph3(selected_dropdown_value1, selected_dropdown_value2):  
    if selected_dropdown_value1 != None:
        df_graph_line = df[df.event_type == selected_dropdown_value1]
    else:df_graph_line = df
    if selected_dropdown_value2 != None:
        df_graph_line_second_update = df_graph_line[df.camera_id == selected_dropdown_value2]
    else: df_graph_line_second_update = df_graph_line
        
    df_graph_line_second_update['datetime'] = pd.to_datetime(df_graph_line_second_update['timestamp'], format='%d%b%Y %H:%M:%S.%f')

    df_graph_line_second_update = df_graph_line_second_update.set_index('datetime')

    df_resampled = df_graph_line_second_update.resample('15Min').count()   
    

    return{
        'data':[go.Scatter(
            x=df_resampled.index,          
            y=df_resampled['timestamp'],   
            text=df_graph_line_second_update['event_type'],
            name= 'IDK',
            type='line',
            mode='lines+markers')
        ],
            'layout': {
                'title' : 'Reporting cameras line'
            }
    }



if __name__ == '__main__':
    app.run_server(debug=False, port=8078)

 * Running on http://127.0.0.1:8078/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_renderer/react@15.4.2.min.js?v=0.11.3 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_renderer/react-dom@15.4.2.min.js?v=0.11.3 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_html_components/bundle.js?v=0.9.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_core_components/plotly-1.33.1.min.js?v=0.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_core_components/bundle.js?v=0.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:22] "GET /_dash-component-suites/dash_renderer/bundle.js?v=0.11.3 HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2018 10:32:23] "GET /_dash-dependencies H